<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/44_4_Pong%2BDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install gym==0.26.2 ale-py==0.8.1

In [4]:
# Установите gym с поддержкой ROM-лицензии
!pip install gym[accept-rom-license] torch numpy opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/

In [7]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random
import os

# Предобработка кадров (убедитесь, что state - numpy array)
def preprocess(state):
    state = state[35:195]  # Обрезка
    state = state[::2, ::2, 0]  # Уменьшение до 80x80
    state[state == 144] = 0  # Удаление фона
    state[state == 109] = 0
    state[state != 0] = 1  # Бинаризация
    return state.astype(np.float32).ravel()

# Архитектура DQN
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# Агент DQN
class DQNAgent:
    def __init__(self, state_dim, action_dim):
        self.q_net = DQN(state_dim, action_dim)
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=1e-3)
        self.memory = deque(maxlen=10000)
        self.batch_size = 32
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return env.action_space.sample()
        state = torch.FloatTensor(state)
        return torch.argmax(self.q_net(state)).item()

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(np.array(states))
        next_states = torch.FloatTensor(np.array(next_states))
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        dones = torch.FloatTensor(dones)

        current_q = self.q_net(states).gather(1, actions.unsqueeze(1))
        next_q = self.q_net(next_states).max(1)[0].detach()
        target_q = rewards + (1 - dones) * self.gamma * next_q

        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# Создание среды
env = gym.make('ALE/Pong-v5', render_mode='rgb_array')
env = gym.wrappers.RecordVideo(env, "pong_videos")

# Параметры
state_dim = 80 * 80
action_dim = env.action_space.n
agent = DQNAgent(state_dim, action_dim)

# Обучение
for episode in range(100):
    state, _ = env.reset()  # Исправлено
    state = preprocess(state)
    total_reward = 0
    done = False

    while not done:
        action = agent.act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)  # Исправлено
        done = terminated or truncated  # Исправлено
        next_state = preprocess(next_state)

        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        agent.replay()

    print(f"Episode: {episode}, Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")

env.close()

/usr/local/lib/python3.11/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/pong_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Moviepy - Building video /content/pong_videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/pong_videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/pong_videos/rl-video-episode-0.mp4
Episode: 0, Reward: -21.0, Epsilon: 0.02
Moviepy - Building video /content/pong_videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/pong_videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/pong_videos/rl-video-episode-1.mp4
Episode: 1, Reward: -21.0, Epsilon: 0.01
Episode: 2, Reward: -21.0, Epsilon: 0.01
Episode: 3, Reward: -21.0, Epsilon: 0.01
Episode: 4, Reward: -21.0, Epsilon: 0.01
Episode: 5, Reward: -21.0, Epsilon: 0.01
Episode: 6, Reward: -20.0, Epsilon: 0.01
Episode: 7, Reward: -21.0, Epsilon: 0.01
Moviepy - Building video /content/pong_videos/rl-video-episode-8.mp4.
Moviepy - Writing video /content/pong_videos/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /content/pong_videos/rl-video-episode-8.mp4
Episode: 8, Reward: -21.0, Epsilon: 0.01
Episode: 9, Reward: -21.0, Epsilon: 0.01
Episode: 10, Reward: -20.0, Epsilon: 0.01
Episode: 11, Reward: -21.0, Epsilon: 0.01
Episode: 12, Reward: -21.0, Epsilon: 0.01
Episode: 13, Reward: -21.0, Epsilon: 0.01
Episode: 14, Reward: -21.0, Epsilon: 0.01
Episode: 15, Reward: -21.0, Epsilon: 0.01
Episode: 16, Reward: -21.0, Epsilon: 0.01
Episode: 17, Reward: -21.0, Epsilon: 0.01
Episode: 18, Reward: -21.0, Epsilon: 0.01
Episode: 19, Reward: -21.0, Epsilon: 0.01
Episode: 20, Reward: -21.0, Epsilon: 0.01
Episode: 21, Reward: -21.0, Epsilon: 0.01
Episode: 22, Reward: -21.0, Epsilon: 0.01
Episode: 23, Reward: -21.0, Epsilon: 0.01
Episode: 24, Reward: -21.0, Epsilon: 0.01
Episode: 25, Reward: -21.0, Epsilon: 0.01
Episode: 26, Reward: -21.0, Epsilon: 0.01
Moviepy - Building video /content/pong_videos/rl-video-episode-27.mp4.
Moviepy - Writing video /content/pong_videos/rl-vi

Moviepy - Done !
Moviepy - video ready /content/pong_videos/rl-video-episode-27.mp4
Episode: 27, Reward: -21.0, Epsilon: 0.01
Episode: 28, Reward: -20.0, Epsilon: 0.01
Episode: 29, Reward: -21.0, Epsilon: 0.01
Episode: 30, Reward: -21.0, Epsilon: 0.01
Episode: 31, Reward: -21.0, Epsilon: 0.01
Episode: 32, Reward: -21.0, Epsilon: 0.01
Episode: 33, Reward: -21.0, Epsilon: 0.01
Episode: 34, Reward: -21.0, Epsilon: 0.01
Episode: 35, Reward: -21.0, Epsilon: 0.01
Episode: 36, Reward: -21.0, Epsilon: 0.01
Episode: 37, Reward: -21.0, Epsilon: 0.01
Episode: 38, Reward: -21.0, Epsilon: 0.01
Episode: 39, Reward: -21.0, Epsilon: 0.01
Episode: 40, Reward: -21.0, Epsilon: 0.01
Episode: 41, Reward: -21.0, Epsilon: 0.01
Episode: 42, Reward: -21.0, Epsilon: 0.01
Episode: 43, Reward: -21.0, Epsilon: 0.01
Episode: 44, Reward: -21.0, Epsilon: 0.01
Episode: 45, Reward: -21.0, Epsilon: 0.01
Episode: 46, Reward: -21.0, Epsilon: 0.01
Episode: 47, Reward: -21.0, Epsilon: 0.01
Episode: 48, Reward: -21.0, Epsilo

Moviepy - Done !
Moviepy - video ready /content/pong_videos/rl-video-episode-64.mp4
Episode: 64, Reward: -21.0, Epsilon: 0.01
Episode: 65, Reward: -21.0, Epsilon: 0.01
Episode: 66, Reward: -21.0, Epsilon: 0.01
Episode: 67, Reward: -21.0, Epsilon: 0.01
Episode: 68, Reward: -21.0, Epsilon: 0.01
Episode: 69, Reward: -21.0, Epsilon: 0.01
Episode: 70, Reward: -21.0, Epsilon: 0.01
Episode: 71, Reward: -21.0, Epsilon: 0.01
Episode: 72, Reward: -21.0, Epsilon: 0.01
Episode: 73, Reward: -21.0, Epsilon: 0.01
Episode: 74, Reward: -21.0, Epsilon: 0.01
Episode: 75, Reward: -21.0, Epsilon: 0.01
Episode: 76, Reward: -21.0, Epsilon: 0.01
Episode: 77, Reward: -21.0, Epsilon: 0.01
Episode: 78, Reward: -21.0, Epsilon: 0.01
Episode: 79, Reward: -21.0, Epsilon: 0.01
Episode: 80, Reward: -21.0, Epsilon: 0.01
Episode: 81, Reward: -21.0, Epsilon: 0.01
Episode: 82, Reward: -21.0, Epsilon: 0.01
Episode: 83, Reward: -21.0, Epsilon: 0.01
Episode: 84, Reward: -21.0, Epsilon: 0.01
Episode: 85, Reward: -21.0, Epsilo

In [12]:
# Вывод видео
from IPython.display import HTML
from base64 import b64encode

video_path = "pong_videos/rl-video-episode-8.mp4"  # Убедитесь, что путь правильный

with open(video_path, "rb") as video_file:
    video_data = video_file.read()

video_base64 = b64encode(video_data).decode()
HTML(f'<video width="1000" height="1000" controls><source src="data:video/mp4;base64,{video_base64}" type="video/mp4"></video>')